# Feature extraction
For each of the experiments, already perform the offline augmentation

In [1]:
%load_ext autoreload
%autoreload 2

from util import config, util
import os
import zipfile
from os.path import join
import random
import pandas as pd
import numpy as np
import shutil
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

## Step 1: Get the raw_feature train and test filenames

In [5]:
raw_feature_train_files_base_path = join(config.RAW_SUBSET_DATA_PATH, 'train/audio')
raw_feature_test_files_base_path = join(config.RAW_SUBSET_DATA_PATH, 'test/audio')

# Create a list of full path filenames of the train and test raw feature files
raw_feature_train_filenames = [join(raw_feature_train_files_base_path, file) for file in os.listdir(raw_feature_train_files_base_path)]
raw_feature_test_filenames = [join(raw_feature_test_files_base_path, file) for file in os.listdir(raw_feature_test_files_base_path)]

## Step 2: Generate the un-augmented train and test features to train the baseline model

In [4]:
# Since we dont want to apply any augmentation, we can just set augment to false
util.generate_and_store_augmented_mel_features(
        raw_feature_train_filenames, 
        config.TRAIN_DATA_BASELINE,
        n_copies=5, 
        augment=False
    )
util.generate_and_store_augmented_mel_features(
        raw_feature_test_filenames, 
        config.TEST_DATA_BASE_PATH, 
        n_copies=1, 
        augment=False
    )

  0%|          | 0/1 [00:00<?, ?it/s]


LibsndfileError: Error opening '/workspaces/project_automated_sound_recognition/data/raw/subset/train/audio': Format not recognised.

## Step 4: Generate the train and test features for the 3 experiments

In [ ]:
# Experiment 1: all augmentation methods

# Set the audiomentations config
audiomentations_config = {
    'augmentations': Compose([
        PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
        TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
        Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
    ])
}

# Set the spec augment configurations
specaugment_config = {
    'configuration': {
        'mask_fraction_min':0.02,
        'mask_fraction_max':0.02,
        'mask_fraction_min_time':0.04,
        'mask_fraction_max_time':0.04,
        'p_spec_augment':0.5,
        'fill_val':0,
        'num_freq_lines':5,
        'num_time_lines':3,
    }
}

util.generate_and_store_augmented_mel_features(
        files_list = raw_feature_train_filenames, 
        result_export_path = join(config.TRAIN_DATA_EX1, 'all_augment'),
        n_copies = config.N_COPIES, 
        augment = True,
        audiomentations_config = audiomentations_config,
        specaugment_config = specaugment_config
    )